# 轮廓     

为了得到轮廓, 需要先获取二值图像.    
在下边的例子中, 我们先使用 canny 边缘检测器获取轮廓, 然后数出一共有几个硬币.     

## 计算轮廓    

In [1]:
import numpy as np
import argparse
import cv2

'''
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True, help = "path to the image")
args = vars(ap.parse_args())
image = cv2.imread(args["image"])
'''

image = cv2.imread("coin.jpg")

gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
blured = cv2.GaussianBlur(gray, (11, 11), 0)
cv2.imshow("image", image)
key = cv2.waitKey(0)

## 计算轮廓
edge =cv2.Canny(blured, 30, 125)
cv2.imshow("edge", edge)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

**Note:**     
- 计算轮廓之前先对图像进行 blur(size = 11) 处理, 是为了更容易检测轮廓.      
- 梯度区间的取值非常关键: 这里的选择原则是任何梯度值小于 30 的被认为是没有边缘, 而梯度值大于 150 的区域被认为是边缘值.   

## 数出硬币的个数     

In [2]:
binary,contours,hierarchy = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print("I count %d coins in this image", len(contours))

coins = image.copy()
cv2.drawContours(coins, contours, -1, (0, 255, 0), 2)

cv2.imshow("Coins", coins)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

I count %d coins in this image 15


**cv2.findContours()函数使用说明:**   
- 返回值为二值图, tuple 形式的轮廓, 层次结构?    
- 因为这个函数会对原图作更改, 因此, 我们传递的第一个参数的原图的拷贝(使用 numpy 的 copy() 函数).    
- cv2.RETR_EXTERNAL 指定我们想要的轮廓类型(外圈轮廓), 使用 cv2.RETR_LIST 可以获取所有轮廓. 使用 cv2.RETR_COMP 和 cv2.RETR_TREE 可以获取层级轮廓.    
- cv2.CHAIN_APPROX_SIMPLE 指定轮廓的拟合方法, 该方法压缩水平的、垂直的和斜的部分，也就是说函数只保留它们的 end point(终点)部分. 这样可以节省计算时间, 提高效率. 如果不适用压缩方法, 可以使用 cv2.CHAIN_APPROX_NONE 参数(不推荐使用).   

**cv2.drawContours()函数使用说明:**   
- 第三个参数表示要绘制的轮廓的索引,因此用户可以绘制单个轮廓. 如果为 -1,表示全部绘制.    

## 将硬币从原图中裁剪出来    

In [3]:
for i, c in enumerate(contours):
    x, y, w, h = cv2.boundingRect(c)
    
    print("coin #%d", (i+1))
    coin = image[y:y+h, x:x+w]
    cv2.imshow("coin", coin)
    
    mask = np.zeros(image.shape[:2], dtype="uint8")
    ((centerX, centerY), radius) = cv2.minEnclosingCircle(c)
    cv2.circle(mask, (int(centerX), int(centerY)), int(radius), 255, -1)
    mask= mask[y:y+h, x:x+w]
    
    cv2.imshow("masked coin", cv2.bitwise_and(coin, coin, mask=mask))
    
key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

coin #%d 1
coin #%d 2
coin #%d 3
coin #%d 4
coin #%d 5
coin #%d 6
coin #%d 7
coin #%d 8
coin #%d 9
coin #%d 10
coin #%d 11
coin #%d 12
coin #%d 13
coin #%d 14
coin #%d 15
